<a href="https://colab.research.google.com/github/tahoeivanova/ML_telegram_chat_bot/blob/main/ML_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
token = 'insert_bot_token'

In [6]:
! pip install pymorphy2
!pip install python-telegram-bot --upgrade


In [7]:
import random
import nltk
import json
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from pymorphy2 import MorphAnalyzer

import pickle

In [8]:
morph = MorphAnalyzer()

def lemmatize_text(text):
  lemmatized_text = []
  for word in text.lower().split():
    lemmatized_text.append(morph.parse(word)[0].normal_form)
  return (' ').join(lemmatized_text)

# lemmatize_text('Хорошего дня')

In [9]:
morph.parse('')[0].normal_form

''

In [10]:
with open('/content/BOT_CONFIG.json') as f:
  BOT_CONFIG = json.load(f)

In [11]:
def clean(text):
  output_text = ''
  for letter in text.lower():
    if letter in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
      output_text+= letter
  return output_text


In [12]:
def get_intent(text):
  # Определяем интент
  for intent in BOT_CONFIG['intents']:
    for example in BOT_CONFIG['intents'][intent]['examples']:
      # расстояние Левинштейна (редакторское расстояние)
      text1 = clean(example)
      text2 = clean(text)
      if len(text1)*len(text2)!=0:
        if nltk.edit_distance(text1, text2)/max(len(text1), len(text1)) < 0.4:
          return intent
  return 'Не удалось определить интент'

In [13]:
def bot(text):
  intent = get_intent_from_model(text)
  return random.choice(BOT_CONFIG['intents'][intent]['responces'])

In [14]:
# def bot(text):
#   intent = get_intent(text)
#   if intent != 'Не удалось определить интент':
#     return random.choice(BOT_CONFIG['intents'][intent]['responces'])
#   else:
#     return 'Извините, я ничего не понял'


In [15]:
# input_text = ''
# while input_text != 'Завершить работу':
#   input_text = input()
#   print(bot(input_text))


### ML

In [16]:
X = []
y = []

for intent in BOT_CONFIG['intents']:
    try:
      if intent != 'other':
        for example in BOT_CONFIG['intents'][intent]['examples']:

            X.append(example)
            y.append(intent)
    except:
      print(BOT_CONFIG['intents'][intent])
      

{'hello': {'examples': ['добрый день', 'всем привет', 'приветствую вас', 'здорова', 'здравствуйте всем', 'мое почтение', 'здравия желаю', 'рад видеть', 'здорово', 'приветствую', 'как поживаешь', 'салам', 'поклон', 'салют', 'здравия желаем', 'здорово живешь', 'хой', 'здравствуйте', 'наше вам', 'добрый день', 'здравствуй', 'наше почтение', 'приветик', 'хай', 'ку', 'шелом', 'здорово живете', 'салам орлы', 'физкульт-привет', 'йоу', 'тевирп', 'аюшки', 'саламчик', 'саламка', 'салютик', 'саламчики', 'опана', 'трямс', 'трям', 'бонжур', 'задрова', 'здарова', 'хаюшки', 'приветец', 'хола', 'халяйки', 'хай-фай', 'превед', 'хелло', 'мабухай', 'здрасти-мордасти'], 'responces': ['здравствуйте!', 'доброго дня.', 'привет :)']}, 'bye': {'examples': ['адью', 'ариведерче', 'ата', 'бай', 'бай-бай', 'будь', 'будь здоров', 'будьте здоровы', 'бывай', 'в добрый час', 'всего', 'всего доброго', 'всего лучшего', 'всего наилучшего', 'всего хорошего', 'всех благ', 'встретимся', 'гудбайте', 'до вечера', 'до встречи'

In [17]:
print(len(X))
print(len(y))
print(X[100])
print(y[100])

1172
1172
как твое имя
whoareyou


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
X_train = [lemmatize_text(text) for text in X_train]
X_test = [lemmatize_text(text) for text in X_test]

In [20]:
vectorizer = CountVectorizer(ngram_range=(2, 4), analyzer='char')
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

print(len(vectorizer.get_feature_names()))
# print(vectorizer.get_feature_names()[100:130])


6016


In [21]:
X_train_vectorized.shape

(937, 6016)

In [22]:
X_train_vectorized.toarray()[0][:31]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

####Logisctic Regression

In [23]:
clf = LogisticRegression().fit(X_train_vectorized, y_train)
# clf = RandomForestClassifier(n_estimators=200).fit(X_train_vectorized, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [24]:
clf.score(X_train_vectorized, y_train)
# 0.8422174840085288 LogRec

0.8313767342582711

In [25]:
clf.score(X_test_vectorized, y_test)
0.2297872340425532 # LogRec

0.2297872340425532

In [34]:
with open('/content/tg_bot_model.pickle', 'wb') as f:
  pickle.dump(clf, f)

with open('/content/tg_bot_vectorizer.pickle', 'wb') as f:
  pickle.dump(vectorizer, f)

In [36]:
with open('/content/tg_bot_model.pickle', 'rb') as f:
  loaded_clf = pickle.load(f)

with open('/content/tg_bot_vectorizer.pickle', 'rb') as f:
  loaded_vectorizer = pickle.load(f)

In [39]:
loaded_clf.predict(loaded_vectorizer.transform(['Как дела']))

array(['howareyou'], dtype='<U25')

In [27]:
len(set(y))

278

In [28]:
clf.predict(vectorizer.transform(['привет']))

array(['hello'], dtype='<U25')

In [29]:
def get_intent_from_model(text):
  return clf.predict(vectorizer.transform([text]))[0]

### Telegram bot

In [31]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    try:
        updater = Updater(token)
    except Exception as err:
      print(err)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [35]:
main()

2021-08-12 23:54:57,380 - apscheduler.scheduler - INFO - Scheduler started
2021-08-12 23:57:20,471 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-08-12 23:57:20,473 - apscheduler.scheduler - INFO - Scheduler has been shut down


### Вызов бота

In [33]:
# input_text = ''
# while input_text != 'Завершить работу':
#   input_text = input()
#   print(bot(input_text))